In [ ]:
import os
import pandas as pd
import numpy as np


def load_excel_files(directory):
    """
    Load all Excel files from the specified directory into a dictionary of DataFrames.
    Returns: A dictionary where keys are file names and values are DataFrames.
    """
    # List Excel files (.xlsx and .xls) in the directory
    excel_files = [f for f in os.listdir(directory) if f.endswith(('.xlsx', '.xls'))]
    data_frames = {}

    for file in excel_files:
        file_path = os.path.join(directory, file)
        try:
            # Read the Excel file into a DataFrame
            df = pd.read_excel(file_path)
            # Store DataFrame in the dictionary with the file name as the key
            data_frames[file] = df
            print(f"Loaded {file} successfully.")
        except Exception as e:
            print(f"Error loading {file}: {e}")

    return data_frames

# Specify the path to the directory containing the files (.xlsx, csv)
directory_path = r'path_to_files_directory'

# Load all files into a dictionary of DataFrames
excel_data = load_excel_files(directory_path)

# Print the keys (file names) of the loaded DataFrames
print("Loaded Excel files:", list(excel_data.keys()))

# Rename the Start Date column to Date, remove 'Original resolutions' text from column names,
# and remove the second column (End Date) from each file.

def clean_dataframe_columns(data_frames):

    for file, df in data_frames.items():
        # Rename 'start date' to 'Date' (case-insensitive)
        new_columns = {col: 'Date' for col in df.columns if col.lower() == 'start date'}
        df.rename(columns=new_columns, inplace=True)
        
        # Remove 'Original resolutions' from any column names
        cleaned_columns = {
            col: col.replace('Original resolutions', '').replace('original resolutions', '').strip()
            for col in df.columns
        }
        df.rename(columns=cleaned_columns, inplace=True)

        # Remove the second column if it exists
        if df.shape[1] > 1:
            df.drop(df.columns[1], axis=1, inplace=True)

        print(f"Processed columns for {file}: {df.columns.tolist()}")
    
    return data_frames

# Process the loaded DataFrames
excel_data_clean = clean_dataframe_columns(excel_data)

In [ ]:
# Loop through each DataFrame in the dictionary and replace '-' with NaN
for file, df in excel_data_clean.items():
    df.replace('-', np.nan, inplace=True)
    df.dropna(axis=1, how='all', inplace=True)  # Drop columns where all values are NaN
    print(f"Replaced '-' with NaN in {file}")

In [ ]:
for file, df in excel_data_clean.items():
    if 'Date' in df.columns:  # Ensure that the column exists
        try:
            print(f"Before conversion, first 5 dates in {file}:")
            print(df['Date'].head())
            
            # Special handling for monthly and yearly files
            if "Month" in file:
                # Expecting format like "Jan 1, 2023"
                df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y', errors='raise')
                # Format as "01/01/2023 00:00"
                df['Date'] = df['Date'].dt.strftime('%d/%m/%Y 00:00')
                print(f"Successfully formatted monthly dates in {file}")
            elif "Year" in file:
                # Expecting format like "Jan 1, 2023"
                df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y', errors='raise')
                # Format as "01/01/2023 00:00"
                df['Date'] = df['Date'].dt.strftime('%d/%m/%Y 00:00')
                print(f"Successfully formatted yearly dates in {file}")
            else:
                # For all other files, expecting a format with time, e.g., "Jan 1, 2023 12:00 AM"
                df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y %I:%M %p', errors='raise')
                df['Date'] = df['Date'].dt.strftime('%d/%m/%Y %H:%M')
                print(f"Successfully formatted dates in {file}")

            print(f"After conversion, first 5 dates in {file}:")
            print(df['Date'].head())
            
        except Exception as e:
            print(f"Error processing 'Date' column in {file}: {e}")
    else:
        print(f"File {file} does not have a 'Date' column.")


In [ ]:

# Remove rows with empty data in all the columns

for file, df in excel_data_clean.items():
    if 'Date' in df.columns:  
        # Drop rows where all columns (EXCEPT "Date") are NaN
        df.dropna(how='all', subset=[col for col in df.columns if col != 'Date'], inplace=True)

        print(f"Cleaned empty rows in {file}. Remaining rows: {len(df)}")

In [ ]:

# Check the data type of the data in the dfs dict

for file, df in excel_data_clean.items():
        print("Data types of columns:")
        print(df.dtypes)


In [ ]:
for file, df in excel_data.items():
        # Exclude the 'Date' column
        cols_to_convert = [col for col in df.columns if col != 'Date']
        

        for col in cols_to_convert:
            # Strip spaces, remove commas, and any non-numeric characters (except for decimals)
            df[col] = df[col].replace({',': '', ' ': '', '€': '', '%': ''}, regex=True)

            # Convert the cleaned column to numeric (float)
            df[col] = pd.to_numeric(df[col], errors='coerce')

        print(f"Cleaned and converted columns to float for {file}")
        print("="*50)

In [ ]:
# Print the names (keys) of all DataFrames in the dictionary
print("List of DataFrames in the dictionary:")
for file_name, df in excel_data.items():
    print(f"- {file_name}: {df.shape} rows x {df.shape[1]} columns")

In [ ]:

#Check for missing values

for file, df in excel_data_clean.items():
    print(f"Checking missing values in: {file}")
    print(df.isnull().sum())  


In [ ]:
for file, df in excel_data_clean.items():
    if 'Date' in df.columns:
        try:
            # Parse the 'Date' column using your specific format.
            df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y %H:%M", dayfirst=True)
        except Exception as e:
            raise ValueError(f"Error parsing dates in {file}: {e}")
        
        # Verify that no dates failed to parse.
        if df['Date'].isna().any():
            raise ValueError(f"Some dates in {file} could not be parsed.")
        
        # Set the 'Date' column as the index
        df.set_index('Date', inplace=True)
    else:
        if not pd.api.types.is_datetime64_any_dtype(df.index):
            print(f"File {file} is missing a valid 'Date' column and its index is not datetime.")
            continue
        else:
            print(f"File {file} already has a datetime index.")

    # Resampling logic based on file naming convention
    if file.endswith('Quarterhour.xlsx'):
        # Aggregate quarter-hourly data to hourly using sum
        df = df.resample('h').sum()  # using lowercase 'h'
        print(f"Aggregated {file} from quarter-hourly to hourly (using sum).")
    elif file.endswith('Year.xlsx') or file.endswith('Month.xlsx'):
        # Disaggregate monthly/yearly data to hourly by forward-filling the value
        df = df.resample('h').ffill()
        print(f"Disaggregated {file} from {file[-8:-4]} to hourly (using forward fill).")
    elif file.endswith('Hour.xlsx'):
        # File is already hourly data; no resampling is needed.
        print(f"Using {file} as hourly data.")
    else:
        print(f"File {file} does not match")
    
    # Interpolate missing numeric values using linear interpolation
    numeric_cols = df.select_dtypes(include='number').columns
    df[numeric_cols] = df[numeric_cols].interpolate(method='linear', axis=0)
    print(f"Interpolated missing values in {file}")
 
    
    # Reset the index so that 'Date' becomes a column again:
    df.reset_index(inplace=True)
    excel_data_clean[file] = df

In [ ]:
# Define a function to determine the suffix based on the file name.
def get_suffix(file_name):
    if "Actual_consumption" in file_name:
        return "_actual_cons"
    elif "Forecasted_consumption" in file_name:
        return "_forecast_cons"
    elif "Actual_generation" in file_name:
        return "_actual_gen"
    elif "Forecasted_generation_Day-Ahead" in file_name:
        return "_forecast_dayahead_gen"
    elif "Generation_Forecast_Intraday" in file_name:
        return "_intraday_gen"
    elif "Automatic_Frequency_Restoration_Reserve" in file_name:
        return "_auto_FRR"
    elif "Manual_Frequency_Restoration_Reserve" in file_name:
        return "_manual_FRR"
    elif "Balancing_energy" in file_name:
        return "_balancing_energy"
    elif "Costs_of_TSOs__without_costs_of_DSOs__" in file_name:
        return "_costs_TSO"
    elif "Cross-border_physical_flows" in file_name:
        return "_cross_border"
    elif "Day-ahead_prices" in file_name:
        return "_dayahead"
    elif "Exported_balancing_services" in file_name:
        return "_exported_balancing"
    elif "Imported_balancing_services" in file_name:
        return "_imported_balancing"
    elif "Frequency_Containment_Reserve" in file_name:
        return "_FCR"
    elif "Installed_generation_capacity" in file_name:
        return "_installed_gen_cap"
    elif "Scheduled_commercial_exchanges" in file_name:
        return "_scheduled_exchanges"
    else:
        return ""

# First, rename columns in each DataFrame (excluding the Date column) based on the file name.
for file, df in excel_data_clean.items():
    suffix = get_suffix(file)
    rename_mapping = {}
    for col in df.columns:
        if col.lower() == 'date':
            continue  
        rename_mapping[col] = col + suffix
    df.rename(columns=rename_mapping, inplace=True)
    

    print(f"Columns in {file} after renaming:")
    print(df.columns.tolist())

# Dictionary to collect columns and the corresponding file names.
columns_files = {}

for file, df in excel_data_clean.items():
    for col in df.columns:
        if col.lower() == 'date':
            continue
        columns_files.setdefault(col, []).append(file)

# Print out columns that appear in more than one file.
print("Columns (excluding 'Date') that appear in multiple files:")
for col, files in columns_files.items():
    if len(files) > 1:
        print(f"Column '{col}' appears in the following files:")
        for f in files:
            print(f"  - {f}")
        print()  


In [ ]:
# Define the output directory
output_dir = r'path_to_output'

# Create the directory if it does not exist
os.makedirs(output_dir, exist_ok=True)


for file, df in excel_data_clean.items():

    processed_filename = f"{file}"
    
    # Define the output path
    output_path = os.path.join(output_dir, processed_filename)
    
    # Save DataFrame as Excel file
    df.to_excel(output_path, index=False)
    
    print(f"Saved {processed_filename} to {output_path}")